In [3]:
from test_compose import combine_images_grid
import autogen
from autogen import Agent, AssistantAgent, ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.llava_agent import LLaVAAgent, llava_call

In [4]:
LLAVA_MODE = "remote"  # Either "local" or "remote"
assert LLAVA_MODE in ["local", "remote"]

In [3]:

if LLAVA_MODE == "remote":
    import replicate

    llava_config_list = [
        {
            "model": "whatever, will be ignored for remote",  # The model name doesn't matter here right now.
            "api_key": "None",  # Note that you have to setup the API key with os.environ["REPLICATE_API_TOKEN"]
            "base_url": "yorickvp/llava-13b:2facb4a474a0462c15041b78b1ad70952ea46b5ec6ad29583c0b29dbd4249591",
        }
    ]

In [6]:
image_compose = autogen.AssistantAgent(
    name='image_compose',
    code_execution_config=False,
    system_message="""
        First you get a compose image from combine_images_grid.
        Our critic agent will give you a feedback and you pass that feedback back to combine_images_grid and get a response image.
    """,
    llm_config={
        "temperature": 0,
        "timeout": 600,
        "cache_seed": 42,
        "config_list": [config.copy() for config in llava_config_list],
        "functions": [
             {
                        "name": "combine_images_grid",
                        "description": "Ask the function to compose images based on the inputs.",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "base_image": {
                                    "type": "array",
                                    "description": "The background image in which the compositon of the other images will be upon."
                                },
                                "overlays": {
                                    "type": "list",
                                    "description": "A list with the image and their respective region in the background image."
                                }
                        }
                    }
              }
        ],
    
    },
)



In [7]:
class AssistantAgent:
    def __init__(self, name, code_execution_config, system_message, llm_config):
        self.name = name
        self.code_execution_config = code_execution_config
        self.system_message = system_message
        self.llm_config = llm_config

    def run(self, function_name, inputs):
        if function_name == "combine_images_grid":
            return combine_images_grid(inputs['base_image'], inputs['overlays'])


image_compose = AssistantAgent(
    name='image_compose',
    code_execution_config=False,
    system_message="""
        First you get a composed image from combine_images_grid.
        Our critic agent will give you feedback, and you pass that feedback back to combine_images_grid to get a response image.
    """,
    llm_config={
        "temperature": 0,
        "timeout": 600,
        "cache_seed": 42,
        "config_list": llava_config_list,  # assuming llava_config_list is defined elsewhere
        "functions": [
            {
                "name": "combine_images_grid",
                "description": "Compose images based on the inputs.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "base_image": {
                            "type": "array",
                            "description": "The background image where the composition will be made."
                        },
                        "overlays": {
                            "type": "list",
                            "description": "A list of overlay images and their positions."
                        }
                    }
                }
            }
        ]
    }
)

In [8]:
image_compose = autogen.AssistantAgent(
    name='image_compose',
    code_execution_config=False,
    system_message="""
        First you get a composed image from combine_images_grid.
        Our critic agent will give you feedback, and you pass that feedback back to combine_images_grid to get a response image.
    """,
    llm_config={
        "temperature": 0,
        "timeout": 600,
        "cache_seed": 42,
        "config_list": llava_config_list,  # assuming llava_config_list is defined elsewhere
        "functions": [
            {
                "name": "combine_images_grid",
                "description": "Compose images based on the inputs.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "base_image": {
                            "type": "array",
                            "description": "The background image where the composition will be made."
                        },
                        "overlays": {
                            "type": "list",
                            "description": "A list of overlay images and their positions."
                        }
                    }
                }
            }
        ]
    }
)

In [9]:
import cv2
# Paths to images
car_image_path = 'data/0a18978cdc8b64f900b0db6a297eb99d/end-copy.jpg'
base_image_path = 'data/0a59be2e7dd53d6de11a10ce3649c081/landing_1.png'
cta_image_path = 'data/0a18978cdc8b64f900b0db6a297eb99d/cta.png'

# Load the images using cv2.imread
base_image = cv2.imread(base_image_path)
cta_image = cv2.imread(cta_image_path)
car_image = cv2.imread(car_image_path)

print(type(base_image))

# Check the loaded images dimensions
print(f"Base image shape: {base_image.shape}")
print(f"CTA image shape: {cta_image.shape}")
print(f"Car image shape: {car_image.shape}")

# Define the overlays with their grid positions
overlays = [
    (cta_image, (0, 2)),  # Extract text from cta_image and place it in the top-right grid cell
    (car_image, (1, 1))   # Extract text from car_image and place it in the center grid cell
]

inputs = {
    'base_image': base_image,
    'overlays': overlays
}

<class 'NoneType'>


[ WARN:0@70.681] global loadsave.cpp:241 findDecoder imread_('data/0a59be2e7dd53d6de11a10ce3649c081/landing_1.png'): can't open/read file: check file path/integrity
[ WARN:0@70.681] global loadsave.cpp:241 findDecoder imread_('data/0a18978cdc8b64f900b0db6a297eb99d/cta.png'): can't open/read file: check file path/integrity
[ WARN:0@70.682] global loadsave.cpp:241 findDecoder imread_('data/0a18978cdc8b64f900b0db6a297eb99d/end-copy.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [8]:
combined_image = image_compose.run("combine_images_grid", inputs)

AttributeError: 'AssistantAgent' object has no attribute 'run'

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(combined_image)

NameError: name 'combined_image' is not defined

In [1]:
lm_config = {"config_list": llava_config_list, "seed": 42}

user_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message="A human admin.",
   code_execution_config={"last_n_messages": 3, "work_dir": "groupchat"},
   human_input_mode="NEVER",
)
# coder = autogen.AssistantAgent(
#     name="Coder",  # the default assistant agent is capable of solving problems with code
#     llm_config={"config_list": llava_config_list, "temperature": 0.5},
# )
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="""
    You are helpful AI with expertise in Adverstiment. You will first identify each component in the composed image
    and you will give feedback based on this information.

    VERTICAL_POSITIONING = {'Logo': [1], 'CTA Button': [1, 2, 3], 'Icon': [1, 2, 3], 'Product Image': [2],
               'Text Elements': [1,3], 'Infographic': [2], 'Banner': [1], 'Illustration': [2], 'Photograph': [2],
               'Mascot': [2], 'Testimonial Quotes': [2], 'Social Proof': [2, 1, 3], 'Seal or Badge': [3, 1, 2],
               'Graphs and Charts': [2], 'Decorative Elements': [3], 'Interactive Elements': [2],
               'Animation': [2], 'Coupon or Offer Code': [3], 'Legal Disclaimers or Terms': [3],
               'Contact Information': [3, 1, 2], 'Map or Location Image': [3], 'QR Code': [3, 1, 2]}

    HORIZONTAL_POSITIONING = {'Logo': [1], 'CTA Button': [2, 1, 3], 'Icon': [1], 'Product Image': [1],
                          'Text Elements': [1], 'Infographic': [1], 'Banner': [2], 'Illustration': [2],
                          'Photograph': [2], 'Mascot': [1], 'Testimonial Quotes': [2], 'Social Proof': [3, 1, 2],
                          'Seal or Badge': [3, 1, 2], 'Graphs and Charts': [1], 'Decorative Elements': [3],
                          'Interactive Elements': [2], 'Animation': [2], 'Coupon or Offer Code': [3],
                          'Legal Disclaimers or Terms': [3], 'Contact Information': [3, 1, 2],
                          'Map or Location Image': [3], 'QR Code': [3, 1, 2]}   

    """,
    llm_config={"config_list": llava_config_list, "temperature": 0.5},
)

groupchat = autogen.GroupChat(agents=[user_proxy, image_compose, critic], messages=[], max_round=20)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config={"config_list": llava_config_list, "temperature": 0.5})


NameError: name 'llava_config_list' is not defined

In [11]:
image_compose = autogen.AssistantAgent(
    name='image_compose',
    code_execution_config=False,
    system_message="""
        First you get a composed image from combine_images_grid.
        Our critic agent will give you feedback, and you pass that feedback back to combine_images_grid to get a response image.
    """,
    llm_config={
        "temperature": 0,
        "timeout": 600,
        "cache_seed": 42,
        "config_list": llava_config_list,  # assuming llava_config_list is defined elsewhere
        "functions": [
            {
                "name": "combine_images_grid",
                "description": "Compose images based on the inputs.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "base_image": {
                            "type": "array",
                            "description": "The background image where the composition will be made."
                        },
                        "overlays": {
                            "type": "list",
                            "description": "A list of overlay images and their positions."
                        }
                    }
                }
            }
        ]
    }
)

In [11]:
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    #is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

In [12]:
llava_config_list

[{'model': 'whatever, will be ignored for remote',
  'api_key': 'None',
  'base_url': 'yorickvp/llava-13b:2facb4a474a0462c15041b78b1ad70952ea46b5ec6ad29583c0b29dbd4249591'}]

In [13]:
chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="For coding tasks, only use the functions you have been provided with.", #Reply TERMINATE when the task is done.",
    llm_config=llava_config_list[0]
)

In [14]:
import cv2
import numpy as np
from typing import Any, List, Tuple
from pydantic import BaseModel

class CombinedImageOutput(BaseModel):
    image_path: str

    class Config:
        arbitrary_types_allowed = True

def resize_to_fit(image: Any, max_height: int, max_width: int) -> np.ndarray:
    """
    Resize an image to fit within the given dimensions while maintaining aspect ratio.

    Parameters:
    image (Any): The image to resize.
    max_height (int): The maximum height of the resized image.
    max_width (int): The maximum width of the resized image.

    Returns:
    np.ndarray: The resized image.
    """
    if not isinstance(image, np.ndarray):
        raise ValueError("Image must be a numpy ndarray.")
    
    h, w = image.shape[:2]
    if h > max_height or w > max_width:
        scaling_factor = min(max_height / h, max_width / w)
        new_size = (int(w * scaling_factor), int(h * scaling_factor))
        return cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)
    return image

@user_proxy.register_for_execution()
@chatbot.register_for_llm(name="python", description="run cell in python and return the image.")
def combine_images_grid(base_image: Any, overlays: List[Tuple[Any, Tuple[int, int]]]) -> CombinedImageOutput:
    """
    Combine multiple overlay images into the base image using a 3x3 grid system.

    Parameters:
    base_image (Any): The base image where overlay images will be placed.
    overlays (list of tuples): List of overlay images and their grid positions. Each tuple should be in the format
                               (overlay_image, (grid_row, grid_col)), where (grid_row, grid_col) specifies the grid cell.

    Returns:
    CombinedImageOutput: The combined image with overlays.
    """
    if not isinstance(base_image, np.ndarray):
        raise ValueError("Base image must be a numpy ndarray.")
    
    # Make a copy of the base image to modify
    combined_image = base_image.copy()
    
    # Get the dimensions of the base image
    base_h, base_w = combined_image.shape[:2]
    
    # Calculate the size of each grid cell
    cell_h, cell_w = base_h // 3, base_w // 3
    
    # Iterate through the overlay images and their grid positions
    for overlay, (grid_row, grid_col) in overlays:
        if not isinstance(overlay, np.ndarray):
            raise ValueError("Overlay images must be numpy ndarrays.")
        
        # Resize the overlay to fit within the grid cell
        overlay = resize_to_fit(overlay, cell_h, cell_w)
        
        # Get the dimensions of the overlay image
        overlay_h, overlay_w = overlay.shape[:2]
        
        # Calculate the top-left corner of the grid cell
        x = grid_col * cell_w
        y = grid_row * cell_h
        
        # Place the overlay image in the center of the grid cell
        center_x = x + (cell_w - overlay_w) // 2
        center_y = y + (cell_h - overlay_h) // 2
        
        # Ensure the region is within the bounds of the base image
        if center_y + overlay_h > base_h or center_x + overlay_w > base_w:
            raise ValueError("Overlay image exceeds the bounds of the base image at the specified location.")
        
        # Place the overlay image on the base image
        combined_image[center_y:center_y+overlay_h, center_x:center_x+overlay_w] = overlay

    # Save the combined image
    image_path = "combined_image.png"
    cv2.imwrite(image_path, combined_image)

    return CombinedImageOutput(image_path=image_path)


In [15]:
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    #is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    #code_execution_config={
     #   "work_dir": "coding",
      #  "use_docker": False,
    #},  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)


In [20]:
car_image_path = 'data/game_endframe.png'
base_image_path = 'data/landing_1 (1).png'
cta_image_path = 'data/cta.png'

# Load the images using cv2.imread
base_image = cv2.imread(base_image_path)
cta_image = cv2.imread(cta_image_path)
car_image = cv2.imread(car_image_path)

print(type(base_image))

# Check the loaded images dimensions
print(f"Base image shape: {base_image.shape}")
print(f"CTA image shape: {cta_image.shape}")
print(f"Car image shape: {car_image.shape}")

# Define the overlays with their grid positions
overlays = [
    (cta_image, (0, 2)),  # Extract text from cta_image and place it in the top-right grid cell
    (car_image, (1, 1))   # Extract text from car_image and place it in the center grid cell
]

<class 'numpy.ndarray'>
Base image shape: (900, 600, 3)
CTA image shape: (64, 300, 3)
Car image shape: (119, 224, 3)


In [17]:
user_agent = autogen.agentchat.register_function(
    combine_images_grid,
    caller=chatbot,
    executor=user_proxy,
    description="Combine multiple images into a grid and save the combined image."
)

/workspaces/Automated-Storyboard-Synthesis-for-Digital-Advertising/venv/lib/python3.10/site-packages/autogen/agentchat/conversable_agent.py:2489: UserWarning: Function 'python' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)


In [21]:
combined_image_output = combine_images_grid(base_image, overlays)

In [22]:
combined_image_output

CombinedImageOutput(image_path='combined_image.png')

In [23]:
critic_agent = autogen.AssistantAgent(
    'Critic',
    system_message="""

    You are an expert in advertisement design and know what constitutes the best ads. Based on the recommended positions for each component, please analyze the provided image and offer feedback on where each component should be placed according to the guidelines below:

    VERTICAL_POSITIONING = {'Logo': [1], 'CTA Button': [1, 2, 3], 'Icon': [1, 2, 3], 'Product Image': [2],
               'Text Elements': [1,3], 'Infographic': [2], 'Banner': [1], 'Illustration': [2], 'Photograph': [2],
               'Mascot': [2], 'Testimonial Quotes': [2], 'Social Proof': [2, 1, 3], 'Seal or Badge': [3, 1, 2],
               'Graphs and Charts': [2], 'Decorative Elements': [3], 'Interactive Elements': [2],
               'Animation': [2], 'Coupon or Offer Code': [3], 'Legal Disclaimers or Terms': [3],
               'Contact Information': [3, 1, 2], 'Map or Location Image': [3], 'QR Code': [3, 1, 2]}

    HORIZONTAL_POSITIONING = {'Logo': [1], 'CTA Button': [2, 1, 3], 'Icon': [1], 'Product Image': [1],
                          'Text Elements': [1], 'Infographic': [1], 'Banner': [2], 'Illustration': [2],
                          'Photograph': [2], 'Mascot': [1], 'Testimonial Quotes': [2], 'Social Proof': [3, 1, 2],
                          'Seal or Badge': [3, 1, 2], 'Graphs and Charts': [1], 'Decorative Elements': [3],
                          'Interactive Elements': [2], 'Animation': [2], 'Coupon or Offer Code': [3],
                          'Legal Disclaimers or Terms': [3], 'Contact Information': [3, 1, 2],
                          'Map or Location Image': [3], 'QR Code': [3, 1, 2]}


    Instructions:

    1. Identify each component in the provided image.
    2. Provide feedback on where each component should be placed based on the vertical and horizontal positioning guidelines. For example, "Move the logo to the top right if it's not there."
    3. Instead of providing the feedback in text, can you provide the region where each component should be. Only givr feedback to identified one.
  """,

  llm_config={
        "temperature": 0,
        "timeout": 600,
        "cache_seed": 42,
        "config_list": llava_config_list,  # assuming llava_config_list is defined elsewhere
        "functions": [
            {
                "name": "combine_images_grid",
                "description": "Compose images based on the inputs.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "base_image": {
                            "type": "array",
                            "description": "The background image where the composition will be made."
                        },
                        "overlays": {
                            "type": "list",
                            "description": "A list of overlay images and their positions."
                        }
                    }
                }
            }
        ]
    }
 )

In [26]:
user_proxy.register_for_execution()(combine_images_grid)

# Create the group chat
groupchat = autogen.GroupChat(
    agents=[critic_agent, user_proxy],
    messages=[],
    max_round=10,
    speaker_selection_method="round_robin",
    enable_clear_history=True,
)

# Create the group chat manager
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llava_config_list[0])

/workspaces/Automated-Storyboard-Synthesis-for-Digital-Advertising/venv/lib/python3.10/site-packages/autogen/agentchat/conversable_agent.py:2408: UserWarning: Function 'python' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


In [24]:
groupchat = autogen.GroupChat(
    agents=[critic_agent, user_proxy],
    messages=[],
    max_round=10,
    speaker_selection_method="round_robin",
    enable_clear_history=True,
    #register_for_execution=True
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llava_config_list[0])

In [25]:
autogen.agentchat.register_function(
    combine_images_grid,
    caller=chatbot,
    executor=groupchat,
    description="Combine multiple images into a grid and save the combined image."
)

AttributeError: 'GroupChat' object has no attribute 'register_for_execution'

In [27]:
import cv2
import numpy as np
from typing import Any, List, Tuple
from pydantic import BaseModel

class CombinedImageOutput(BaseModel):
    image_path: str

    class Config:
        arbitrary_types_allowed = True

def resize_to_fit(image: Any, max_height: int, max_width: int) -> np.ndarray:
    if not isinstance(image, np.ndarray):
        raise ValueError("Image must be a numpy ndarray.")
    
    h, w = image.shape[:2]
    if h > max_height or w > max_width:
        scaling_factor = min(max_height / h, max_width / w)
        new_size = (int(w * scaling_factor), int(h * scaling_factor))
        return cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)
    return image

def combine_images_grid(base_image: Any, overlays: List[Tuple[Any, Tuple[int, int]]]) -> CombinedImageOutput:
    if not isinstance(base_image, np.ndarray):
        raise ValueError("Base image must be a numpy ndarray.")
    
    combined_image = base_image.copy()
    base_h, base_w = combined_image.shape[:2]
    cell_h, cell_w = base_h // 3, base_w // 3
    
    for overlay, (grid_row, grid_col) in overlays:
        if not isinstance(overlay, np.ndarray):
            raise ValueError("Overlay images must be numpy ndarrays.")
        
        overlay = resize_to_fit(overlay, cell_h, cell_w)
        overlay_h, overlay_w = overlay.shape[:2]
        x = grid_col * cell_w
        y = grid_row * cell_h
        center_x = x + (cell_w - overlay_w) // 2
        center_y = y + (cell_h - overlay_h) // 2
        
        if center_y + overlay_h > base_h or center_x + overlay_w > base_w:
            raise ValueError("Overlay image exceeds the bounds of the base image at the specified location.")
        
        combined_image[center_y:center_y+overlay_h, center_x:center_x+overlay_w] = overlay

    image_path = "combined_image.png"
    cv2.imwrite(image_path, combined_image)

    return CombinedImageOutput(image_path=image_path)


In [12]:
import autogen

# Define llm_config (assuming llava_config_list is defined elsewhere)
llm_config = {
    "temperature": 0,
    "timeout": 600,
    "cache_seed": 42,
    "config_list": openai_config_list,  # Ensure this is defined
    "functions": [
        {
            "name": "combine_images_grid",
            "description": "Compose images based on the inputs.",
            "parameters": {
                "type": "object",
                "properties": {
                    "base_image": {
                        "type": "array",
                        "description": "The background image where the composition will be made."
                    },
                    "overlays": {
                        "type": "list",
                        "description": "A list of overlay images and their positions."
                    }
                }
            }
        }
    ]
}

# Create UserProxyAgent
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    llm_config=llm_config  # Set the llm_config
)

# Create Critic Agent
critic_agent = autogen.AssistantAgent(
    'Critic',
    system_message="""
    You are an expert in advertisement design and know what constitutes the best ads. Based on the recommended positions for each component, please analyze the provided image and offer feedback on where each component should be placed according to the guidelines below:

    VERTICAL_POSITIONING = {'Logo': [1], 'CTA Button': [1, 2, 3], 'Icon': [1, 2, 3], 'Product Image': [2],
               'Text Elements': [1,3], 'Infographic': [2], 'Banner': [1], 'Illustration': [2], 'Photograph': [2],
               'Mascot': [2], 'Testimonial Quotes': [2], 'Social Proof': [2, 1, 3], 'Seal or Badge': [3, 1, 2],
               'Graphs and Charts': [2], 'Decorative Elements': [3], 'Interactive Elements': [2],
               'Animation': [2], 'Coupon or Offer Code': [3], 'Legal Disclaimers or Terms': [3],
               'Contact Information': [3, 1, 2], 'Map or Location Image': [3], 'QR Code': [3, 1, 2]}

    HORIZONTAL_POSITIONING = {'Logo': [1], 'CTA Button': [2, 1, 3], 'Icon': [1], 'Product Image': [1],
                          'Text Elements': [1], 'Infographic': [1], 'Banner': [2], 'Illustration': [2],
                          'Photograph': [2], 'Mascot': [1], 'Testimonial Quotes': [2], 'Social Proof': [3, 1, 2],
                          'Seal or Badge': [3, 1, 2], 'Graphs and Charts': [1], 'Decorative Elements': [3],
                          'Interactive Elements': [2], 'Animation': [2], 'Coupon or Offer Code': [3],
                          'Legal Disclaimers or Terms': [3], 'Contact Information': [3, 1, 2],
                          'Map or Location Image': [3], 'QR Code': [3, 1, 2]}

    Instructions:
    1. Identify each component in the provided image.
    2. Provide feedback on where each component should be placed based on the vertical and horizontal positioning guidelines. For example, "Move the logo to the top right if it's not there."
    3. Instead of providing the feedback in text, can you provide the region where each component should be. Only give feedback to identified ones.
    """,
    llm_config=llm_config  # Set the llm_config
)

# Create Group Chat
groupchat = autogen.GroupChat(
    agents=[critic_agent, user_proxy],
    messages=[],
    max_round=10,
    speaker_selection_method="round_robin",
    enable_clear_history=True
)

# Register the function with the user proxy agent
user_proxy.register_for_llm(name="combine_images_grid", description="Combine multiple images into a grid and save the combined image.")(combine_images_grid)

# Initialize the Group Chat Manager
manager = autogen.GroupChatManager(groupchat=groupchat)

In [13]:
manager.initiate_chat(
        chatbot,
        message="Draw two agents chatting with each other with an example dialog.",
        max_turns=3
)

chat_manager (to chatbot):

Draw two agents chatting with each other with an example dialog.

--------------------------------------------------------------------------------


BadRequestError: Error code: 400 - {'error': {'message': "Invalid schema for function 'python': In context=('properties', 'overlays', 'items'), array schema missing items", 'type': 'invalid_request_error', 'param': 'tools[0].function.parameters', 'code': 'invalid_function_parameters'}}

In [46]:
manager.inititchat()

AttributeError: 'GroupChatManager' object has no attribute 'run'

In [2]:
#from test_compose import combine_images_grid
import autogen
from autogen import Agent, AssistantAgent, ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.llava_agent import LLaVAAgent, llava_call
import os

LLAVA_MODE = "remote"  # Either "local" or "remote"
assert LLAVA_MODE in ["local", "remote"]

/workspaces/Automated-Storyboard-Synthesis-for-Digital-Advertising/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
if LLAVA_MODE == "remote":

    llava_config_list = [
        {
            "model": "whatever, will be ignored for remote",  # The model name doesn't matter here right now.
            "api_key": os.environ["REPLICATE_API_TOKEN"],  # Note that you have to setup the API key with os.environ["REPLICATE_API_TOKEN"]
            "base_url": "yorickvp/llava-13b:2facb4a474a0462c15041b78b1ad70952ea46b5ec6ad29583c0b29dbd4249591",  # Correctly formatted URL
        }
    ]

In [4]:
openai_config_list = [
    {
        'model': 'gpt-3.5-turbo',
        'api_key': os.getenv('OPENAI_API_KEY'),
    },
]

In [6]:
llm_llava_config = {
    "config_list": llava_config_list,
    "timeout": 120,
}

llm_openai_config = {
    "config_list": openai_config_list,
    "timeout": 120,
}

chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="For coding tasks, only use the functions you have been provided with.",
    llm_config=llm_openai_config,
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

In [7]:
import cv2
import numpy as np
from typing import Any, List, Tuple
from pydantic import BaseModel

class CombinedImageOutput(BaseModel):
    image_path: str

    class Config:
        arbitrary_types_allowed = True

def resize_to_fit(image: Any, max_height: int, max_width: int) -> np.ndarray:
    """
    Resize an image to fit within the given dimensions while maintaining aspect ratio.

    Parameters:
    image (Any): The image to resize.
    max_height (int): The maximum height of the resized image.
    max_width (int): The maximum width of the resized image.

    Returns:
    np.ndarray: The resized image.
    """
    if not isinstance(image, np.ndarray):
        raise ValueError("Image must be a numpy ndarray.")
    
    h, w = image.shape[:2]
    if h > max_height or w > max_width:
        scaling_factor = min(max_height / h, max_width / w)
        new_size = (int(w * scaling_factor), int(h * scaling_factor))
        return cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)
    return image

@user_proxy.register_for_execution()
@chatbot.register_for_llm(name="python", description="run cell in python and return the image.")
def combine_images_grid(base_image: Any, overlays: List[Tuple[Any, Tuple[int, int]]]) -> CombinedImageOutput:
    """
    Combine multiple overlay images into the base image using a 3x3 grid system.

    Parameters:
    base_image (Any): The base image where overlay images will be placed.
    overlays (list of tuples): List of overlay images and their grid positions. Each tuple should be in the format
                               (overlay_image, (grid_row, grid_col)), where (grid_row, grid_col) specifies the grid cell.

    Returns:
    CombinedImageOutput: The combined image with overlays.
    """
    if not isinstance(base_image, np.ndarray):
        raise ValueError("Base image must be a numpy ndarray.")
    
    # Make a copy of the base image to modify
    combined_image = base_image.copy()
    
    # Get the dimensions of the base image
    base_h, base_w = combined_image.shape[:2]
    
    # Calculate the size of each grid cell
    cell_h, cell_w = base_h // 3, base_w // 3
    
    # Iterate through the overlay images and their grid positions
    for overlay, (grid_row, grid_col) in overlays:
        if not isinstance(overlay, np.ndarray):
            raise ValueError("Overlay images must be numpy ndarrays.")
        
        # Resize the overlay to fit within the grid cell
        overlay = resize_to_fit(overlay, cell_h, cell_w)
        
        # Get the dimensions of the overlay image
        overlay_h, overlay_w = overlay.shape[:2]
        
        # Calculate the top-left corner of the grid cell
        x = grid_col * cell_w
        y = grid_row * cell_h
        
        # Place the overlay image in the center of the grid cell
        center_x = x + (cell_w - overlay_w) // 2
        center_y = y + (cell_h - overlay_h) // 2
        
        # Ensure the region is within the bounds of the base image
        if center_y + overlay_h > base_h or center_x + overlay_w > base_w:
            raise ValueError("Overlay image exceeds the bounds of the base image at the specified location.")
        
        # Place the overlay image on the base image
        combined_image[center_y:center_y+overlay_h, center_x:center_x+overlay_w] = overlay

    # Save the combined image
    image_path = "combined_image.png"
    cv2.imwrite(image_path, combined_image)

    return CombinedImageOutput(image_path=image_path)


In [8]:
autogen.agentchat.register_function(
    combine_images_grid,
    caller=chatbot,
    executor=user_proxy,
   # name="sh",
    description="run a python script and return the execution result.",
)

/workspaces/Automated-Storyboard-Synthesis-for-Digital-Advertising/venv/lib/python3.10/site-packages/autogen/agentchat/conversable_agent.py:2489: UserWarning: Function 'python' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/workspaces/Automated-Storyboard-Synthesis-for-Digital-Advertising/venv/lib/python3.10/site-packages/autogen/agentchat/conversable_agent.py:2408: UserWarning: Function 'python' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


In [9]:
user_proxy.initiate_chat(
        chatbot,
        message="Draw two agents chatting with each other with an example dialog.",
        max_turns=3
)

user_proxy (to chatbot):

Draw two agents chatting with each other with an example dialog.

--------------------------------------------------------------------------------


BadRequestError: Error code: 400 - {'error': {'message': "Invalid schema for function 'python': In context=('properties', 'overlays', 'items'), array schema missing items", 'type': 'invalid_request_error', 'param': 'tools[0].function.parameters', 'code': 'invalid_function_parameters'}}